### Calculating the sensitivity of SSAM output to small fluctuations in input
Quantifying differences in the output files of Salish Sea Atlantis Model simulations with small differences in input

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import ssam_groups as groups
import ssam_plot as splot

In [2]:
def my_rmsd_percent(one, two):
    one_all_layers = one.sum(axis=2)
    two_all_layers = two.sum(axis=2)
    diff = abs(one_all_layers-two_all_layers)
    diff_ratio = diff/two
    diff_ratio_squared = diff_ratio**2
    diff_ratio_squared_mean1 = diff_ratio_squared.mean(axis=1, skipna=True)
    diff_ratio_squared_mean2 = diff_ratio_squared_mean1.mean(axis=0, skipna=True)
    my_rmsd_percent = ((diff_ratio_squared_mean2)**(1/2))*100
    return my_rmsd_percent

In [15]:
scenario_path = Path('/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/')

control_file = "/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/control/outputSalishSea.nc"
control= xr.open_dataset(str(control_file), decode_cf=True)
annual_control = control.resample(t='5Y').mean()

In [4]:
# Masking land boxes
Temps = np.ma.filled(control.Temp[1,:,5], np.nonzero)
(ocean_boxes) = Temps.nonzero()
ocean_boxes = ocean_boxes[0]
ocean_boxes = ocean_boxes[1:ocean_boxes.size-1]
ocean_boxes

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,  14,
        15,  16,  18,  19,  20,  21,  23,  26,  28,  31,  32,  33,  34,
        35,  36,  37,  38,  39,  40,  54,  55,  57,  58,  60,  62,  63,
        64,  66,  68,  72,  73,  74,  75,  79,  80,  81,  82,  83,  84,
        85,  86,  88,  90,  91,  92,  94,  96,  98, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124])

In [33]:
# test new equation
scenario_test_path = '/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/butterflyeffect_MAx11/outputSalishSea.nc'
scenario_test = xr.open_dataset(str(scenario_test_path), decode_cf=True)

one = control.variables['Diatom_N'][0:142][ocean_boxes][:]
two = scenario_test.variables['Diatom_N'][:][ocean_boxes][:]

In [34]:
my_rmsd_percent(one, two)

TypeError: 'Variable' object is not callable

In [38]:
#file_dict = {'BC', 'FHE', 'MA', 'ORR', 'PL', 'PS', 'WHB'}
file_dict = {'MA', 'PS'}

scens = sorted([p for p in scenario_path.glob('butterflyeffect_*x11/outputSalishSea.nc')])

# measured opening of scenario files
for path in scens:
    nm = str(path).split(sep = '/')
    name1 = nm[7].split(sep='_')
    name = name1[1]

    rmsd_data = pd.DataFrame({'scenario_file': [],
                    'species': [],
                    'RMSD%': []},)
    scenario = xr.open_dataset(str(path), decode_cf=True)
    annual_scenario = scenario.resample(t='5Y').mean()
    for species in groups.all_pelagic:
        two = annual_scenario.variables[groups.all_pelagic[species] +'_N'][:][ocean_boxes]
        one = annual_control.variables[groups.all_pelagic[species] + '_N'][0:142][ocean_boxes]

        rmsd_score = my_rmsd_percent(one, two)
        rmsd_data.loc[len(rmsd_data.index)] = [name, species, float(rmsd_score)]                       
    rmsd_data.to_csv("/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/rmsd_percent_"+name+"_13Jul.csv")